# Imports




In [3]:
from google.colab import drive
drive.mount('/content/drive')
!pip install ipynb
import sys
sys.path.insert(0,'/content/drive/MyDrive/CNN-deterministic-decoupling')
path = '/content/drive/MyDrive/CNN-deterministic-decoupling'
import numpy as np
import torch
from ipynb.fs.defs.imageFunctions import prepare_image,show_image,save_image
from ipynb.fs.defs.DISTS_mod import DISTS
from PIL import Image
import matplotlib.pyplot as plt
import math

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Transfer

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ref = prepare_image(Image.open(path+"/imagenes/texture5.jpg").convert("RGB"),size=128)
ref=ref.to(device)
ref.requires_grad_(False)
show_image(ref)

#random=torch.normal(mean=0.45, std=0.22,size=ref.size())
origin=prepare_image(Image.open(path+"/imagenes/texture4.jpg").convert("RGB"),size=128)
origin=origin.to(device)
origin.requires_grad_(True)

x0=origin.detach().clone()
x0=x0.to(device)



LossList=[]

model_d = DISTS().to(device)
model_d.set_target(ref)
count = 0



initial_loss=model_d.forward(origin).item()
LossList=[initial_loss]
last_error=LossList[-1]
count+=1

optimizer = torch.optim.Adam([{"params": [origin],"lr": 0.001}])


while True:

    last_error=LossList[-1]
    optimizer.zero_grad()
    loss=model_d.forward(origin)
    loss.backward()
    optimizer.step()

    LossList.append(loss.item())
    count+=1

    if count%2000==0:
      print("iteracion "+str(count))
      print(last_error)
      show_image(origin)
      save_image(origin,path+'/Datos/Adam_t_4_5.png')
      with open(path+'/Datos/Adam_t_4_5.txt', 'w') as f:
        f.write(f'Total iterations: {count}\n')
        f.write('\n')
        f.write(f'Loss values: {LossList}\n')


print("END 1")
print("iteracion "+str(count))
save_image(origin,path+'/Datos/Adam_t_4_5.png')
show_image(origin)

In [ ]:
save_image(origin,path+'/Datos/Adam_t_4_5.png')

In [ ]:
plt.plot(EpochList,LossList)
plt.grid(True)

In [ ]:
plt.plot(EpochList[:200],LossList[:200])
plt.grid(True)

In [ ]:
plt.plot(LossList,ErrorList)
plt.grid(True)

In [ ]:
show_image(origin)